In [12]:
%%html
<script>
    $('#header').toggle()
</script>

In [1]:
import ipywidgets as widgets
import json
import pandas as pd
import requests
from IPython.display import display, HTML, IFrame
from pyvis.network import Network

pd.options.display.max_rows = None

In [2]:
layout_main = {
    'width': '100%',
    'height': '100%',
}

In [3]:
image_header = widgets.Image(
    value=open('images/DI_NLP_Header.png', 'rb').read(),
    format='png',
    layout=layout_main,
)

In [4]:
input_url = widgets.Text(
    placeholder='TextCat URL',
    layout=layout_main,
)
input_text = widgets.Textarea(
    placeholder='Text for analysis',
    layout=layout_main,
)

In [5]:
output_timer = widgets.Output()
output_text = widgets.Output()
output_graph = widgets.Output()
output_table = widgets.Output(
    layout={
        'width': '3000px',
        'height': '100%',
    },
)
output_group = widgets.Tab(
    children=[
        output_text,
        output_graph,
        output_table,
    ],
    layout=layout_main,
)
output_group.set_title(0, 'Text')
output_group.set_title(1, 'Graph')
output_group.set_title(2, 'Table')

In [6]:
@output_timer.capture(clear_output=True, wait=True)
def set_output_timer(data=[]):
    if not data:
        display(HTML('Error'))
    else:
        display(HTML('Done in: ' + str(data) + ' s'))

@output_text.capture(clear_output=True, wait=True)
def set_output_text(data=[]):
    if not data:
        display(HTML('Error'))
    else:
        display(HTML(data))

@output_graph.capture(clear_output=True, wait=True)
def set_output_graph(data=[], keys_display_order=[]):
    if not data:
        display(HTML('Error'))
    else:
        g_pyvis = Network(
            notebook=True,
            width='100%',
            height='100%',
        )
        g_pyvis.add_nodes(
            [ent['ent'] for ent in data],
            color=[ent['colour'] for ent in data],
            title=[
                pd.DataFrame(
                    [(key[1], ent[key[0]])
                     for key in keys_display_order
                     if key[0] in ent.keys()],
                ).style.set_table_styles([
                    {'selector': 'th.col_heading',
                     'props': [('display', 'none')]},
                    {'selector': 'th.row_heading',
                     'props': [('display', 'none')]},
                ]).render()
                for ent in data
            ],
        )
        g_pyvis.show('images/DI_NLP_Graph.html')
        display(
            IFrame(
                'images/DI_NLP_Graph.html',
                width='100%',
                height='1000px',
            ),
        )

@output_table.capture(clear_output=True, wait=True)
def set_output_table(data=[], keys_display_order=[]):
    if not data:
        display(HTML('Error'))
    else:
        df = pd.DataFrame(
            [[ent[key[0]] if key[0] in ent.keys() else ''
              for key in keys_display_order]
             for ent in sorted(data, key=lambda k:k['ent_type'])],
            columns=[key[1] for key in keys_display_order],
        )
        df = df.style.set_table_styles([
            {'selector': 'td',
             'props': [('text-align', 'left')]},
            {'selector': 'th.col_heading',
             'props': [('text-align', 'left')]},
            {'selector': 'th.row_heading',
             'props': [('text-align', 'right')]},
        ])
        display(df)

In [7]:
def textcat(dummy):
    if input_url.value and input_text.value:
        r = requests.post(
            input_url.value,
            data=json.dumps({'text': input_text.value}),
            params={'return_app': 'true'},
            headers={'Content-Type': 'application/json'},
        )
        if r.ok and r.json():
            set_output_timer(r.json()['timer'])
            set_output_text(r.json()['app']['html'])
            set_output_graph(r.json()['ents'], r.json()['app']['keys_display_order'])
            set_output_table(r.json()['ents'], r.json()['app']['keys_display_order'])
        else:
            set_output_timer()
            set_output_text()
            set_output_graph()
            set_output_table()

def clear(dummy):
    input_text.value=''
    output_timer.clear_output()
    output_text.clear_output()
    output_graph.clear_output()
    output_table.clear_output()

In [8]:
button_go = widgets.Button(
    description='Go',
    tooltip='Click to analyse text',
    layout={
        'width': '100px',
    },
)
button_clear = widgets.Button(
    description='Clear',
    tooltip='Click to clear input and output',
    layout={
        'width': '100px',
    },
)
button_go.on_click(textcat)
button_clear.on_click(clear)

In [10]:
VBox_main = widgets.VBox(
    children=[
        image_header,
        input_url,
        input_text,
        widgets.HBox(
            children=[
                button_go,
                button_clear,
                output_timer,
            ],
        ),
        output_group,
    ],
    layout={
        'width': '90%',
        'left': '10%',
        'padding': '5px',
#         'border': '1px solid black',
    },
)
display(VBox_main)